In [1]:
import sys  
sys.path.insert(0, '/home/yon/jupyter-server/mlg/src/')

import config
import loader
import model as m
import preprocessing

# if u want to use the big config
#config = config.big

In [1]:
# temporary imports (delete later)
import sys  
sys.path.insert(0, '/home/yon/jupyter-server/mlg/src/')

import loader
import config
import model as M
import preprocessing
from pprint import pprint
import torch
import random
import torch_geometric
import numpy as np
import time
import matplotlib.pyplot as plt
import pickle
import torch_geometric.transforms as T
import os
import json
import networkx as nx
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
!ls spotify_million_playlist_dataset/pickles

carloss62.pkl	      ghetero_example.pkl     top-ghetero-1000.pkl
carloss66.pkl	      ghetero.pkl	      top-ghetero-5000-fixed-maybe.pkl
carloss72.pkl	      G.pkl		      top-ghetero-5000.pkl
datasets_example.pkl  index.pkl		      top-idx-1000.pkl
datasets.pkl	      top-G-1000_example.pkl  top-idx-5000.pkl
deleteme.pkl	      top-G-1000.pkl
G_example.pkl	      top-G-5000.pkl


In [4]:
converter = pickle.load(open(config.pickles + "/top-idx-1000.pkl", "rb"))

In [6]:
converter.keys()

dict_keys(['playlist', 'track', 'artist', 'album'])

In [20]:
index = {
    "playlist": {},
    "track": {},
    "artist": {},
    "album": {},
}

dataset_path = config.big.dataset_path
filenames = os.listdir(dataset_path)
for i in tqdm(range(len(filenames)), unit="files"):
    with open(os.path.join(dataset_path, filenames[i])) as json_file:
        playlists = json.load(json_file)["playlists"]
        for playlist in playlists:
            playlist_name = f"spotify:playlist:{playlist['pid']}"
            index["playlist"][playlist_name] = playlist["name"]
            for track in playlist["tracks"]:
                index["track"][track["track_uri"]] = track["track_name"]
                index["artist"][track["artist_uri"]] = track["artist_name"]
                index["album"][track["album_uri"]] = track["album_name"]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [06:32<00:00,  2.55files/s]


In [4]:
index = pickle.load(open(config.pickles + "/index.pkl", "rb"))

In [5]:
index.keys()

dict_keys(['playlist', 'track', 'artist', 'album'])

In [7]:
list(index["playlist"].keys())[:5]

['spotify:playlist:549000',
 'spotify:playlist:549001',
 'spotify:playlist:549002',
 'spotify:playlist:549003',
 'spotify:playlist:549004']

In [15]:
index2 = {key: {} for key in index.keys()}
for typ in index.keys():
    for key in index[typ].keys():
        key2 = key.split(":")[2]
        index2[typ][key2] = index[typ][key]

In [17]:
pickle.dump(index2, open(config.pickles + "/index.pkl", "wb"))

In [25]:
pickle.dump(index, open(config.pickles + "/index.pkl", "wb"))

In [5]:
G = pickle.load(open(config.pickles + "/G.pkl", "rb"))
preprocessing.smart_split(G, splits=[5000], pickle_location=config.pickled_top_G)

[5000] started
[5000] got top n playlists in 1.4853570461273193 seconds
[5000] finshed getting neighbors in 5315.407293796539 seconds (288032 nodes))
[5000] finished subgraphing in 43.11258053779602 seconds
[5000] finished pickling in 0.982130765914917 seconds


In [6]:
# G = loader.get_g()
# preprocessing.smart_split(G, splits=[100], pickle_location=lambda _:config.pickles + "/deleteme.pkl")
topG = pickle.load(open(config.pickles + "/top-G-5000_example.pkl", "rb"))
ghetero = loader.nx2hetero(topG, config.pickles + "/top-idx-5000.pkl")
pickle.dump(ghetero, open(config.pickles + "/top-ghetero-5000.pkl", "wb"))

Saved node index to spotify_million_playlist_dataset/pickles/top-idx-5000.pkl


In [6]:
ghetero = loader.get_data()
data_train, data_val, data_test = loader.get_datasets()
ghetero = ghetero.to(device)
data_train = data_train.to(device)

Loading ghetero from pickle ...
Loading datasets from pickle ...


In [8]:
class LinkNeighborLoader(torch_geometric.loader.LinkLoader):
    def __init__(
        self,
        data,
        num_neighbors,
        edge_label_index = None,
        edge_label = None,
        edge_label_time = None,
        replace = False,
        directed = True,
        disjoint = False,
        temporal_strategy = 'uniform',
        neg_sampling = None,
        neg_sampling_ratio = None,
        time_attr = None,
        transform = None,
        is_sorted = False,
        filter_per_worker = False,
        neighbor_sampler = None,
        **kwargs,
    ):
        # TODO(manan): Avoid duplicated computation (here and in NodeLoader):
        edge_type, _ = torch_geometric.loader.utils.get_edge_label_index(data, edge_label_index)

        if (edge_label_time is not None) != (time_attr is not None):
            raise ValueError("I fart")

        if neighbor_sampler is None:
            neighbor_sampler = torch_geometric.sampler.NeighborSampler(
                data,
                num_neighbors=num_neighbors,
                replace=replace,
                directed=directed,
                disjoint=disjoint,
                temporal_strategy=temporal_strategy,
                input_type=edge_type,
                time_attr=time_attr,
                is_sorted=is_sorted,
                share_memory=kwargs.get('num_workers', 0) > 0,
            )

        super().__init__(
            data=data,
            link_sampler=neighbor_sampler,
            edge_label_index=edge_label_index,
            edge_label=edge_label,
            edge_label_time=edge_label_time,
            neg_sampling=neg_sampling,
            neg_sampling_ratio=neg_sampling_ratio,
            transform=transform,
            filter_per_worker=filter_per_worker,
            **kwargs,
        )


In [11]:
class Impostor(torch.nn.Module):
    """Intelligent Mmusic Prediction Operating System with Top-level Optimization and Recommendation."""

    def __init__(self, data_train):
        super().__init__()
        self.model = m.HeteroModel(64, ghetero.x_dict, ghetero.metadata())
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01)

        edge_label_index = data_train["track", "contains", "playlist"].edge_label_index

        edge_label = data_train["track", "contains", "playlist"].edge_label

        edge_label_index = edge_label_index.to(device)
        data_train = data_train.to(device)

        g = torch.Generator(device="cuda")
        g.manual_seed(0)
        self.train_loader = LinkNeighborLoader(
            data=data_train,
            num_neighbors=[20, 10],
            neg_sampling_ratio=2.0,
            edge_label_index=(("track", "contains", "playlist"), edge_label_index),
            edge_label=edge_label,
            batch_size=128,
            shuffle=True,
            generator=g,
            transform_fn=lambda x: x.to(device),
        )

    def forward(self, data):
        return self.model.forward(data)

    def train(self):
        self.model.train()

        total_examples = total_loss = 0
        for batch in self.train_loader:
            self.optimizer.zero_grad()
            batch_size = 100
            out = self.model(batch)
            loss = torch.nn.functional.cross_entropy(
                out, batch["track", "contains", "playlist"].edge_label
            )
            loss.backward()
            self.optimizer.step()

            total_examples += batch_size
            print(f'Loss: {loss:.4f}')
            total_loss += float(loss) * batch_size

        return total_loss / total_examples

# create training mask for playlist nodes
train_mask = torch.zeros(ghetero["playlist"].x.shape[0], dtype=torch.bool)
train_mask[torch.randperm(train_mask.shape[0])[:int(train_mask.shape[0]*0.8)]] = True
train_mask = train_mask.to(device)

ghetero["playlist"].train_mask = train_mask

ghetero["playlist"].y = torch.LongTensor([1]*ghetero["playlist"].x.shape[0])

model = Impostor(data_train)
# model = model.to(device)
model.train()

TypeError: DataLoader.__init__() got an unexpected keyword argument 'transform_fn'